# Pemodelan Machine Learning untuk Memprediksi Level Kemacetan di Kota Bandung

<div align="center">
<img src="https://i.imgur.com/P5PeqCs.jpg">
</div>


# Inisialisasi

In [97]:
!pip install geopandas

In [98]:
!pip install rfpimp

In [99]:
import pandas as pd
import numpy as np
from datetime import datetime
import geopandas
import json
from shapely.geometry import LineString, MultiLineString
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, accuracy_score

df = pd.read_csv('drive/MyDrive/Traffic_Jams.csv')
df.head(5)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,avg_length,avg_delay,avg_speed_kmh,total_records,id,date,geometry
0,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Van Deventer,3,176.0000,106.0000,4.920000,1,11847,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.61..."
1,2019-05-31 23:00:00,32.73,KOTA BANDUNG,HOS Cokroaminoto,3,307.0000,144.0000,6.130000,1,14892,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.59..."
2,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Sukaati,2,207.0000,60.0000,6.110000,1,11803,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.62..."
3,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Lama,3,220.0000,161.5000,3.931000,10,11941,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[..."
4,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Raya,3,1453.4839,475.6129,8.523226,31,11942,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[..."


In [100]:
list(df.columns)

['time',
 'kemendagri_kabupaten_kode',
 'kemendagri_kabupaten_nama',
 'street',
 'level',
 'avg_length',
 'avg_delay',
 'avg_speed_kmh',
 'total_records',
 'id',
 'date',
 'geometry']

In [101]:
df.shape

(136807, 12)

# Exploratory Data Analysis

## Data Wrangling

In [102]:
df.isnull().mean()

# Tidak ditemukan missing value di semua fitur kecuali di fitur street. Tidak akan dilakukan treatment untuk missing value 
# tersebut karena fitur street akan digantikan oleh fitur sub wilayah kota yang akan diperoleh menggunakan data eksternal.

time                         0.000000
kemendagri_kabupaten_kode    0.000000
kemendagri_kabupaten_nama    0.000000
street                       0.007916
level                        0.000000
avg_length                   0.000000
avg_delay                    0.000000
avg_speed_kmh                0.000000
total_records                0.000000
id                           0.000000
date                         0.000000
geometry                     0.000000
dtype: float64

## Transformasi tanggal ke jam dan hari

In [103]:
# Saya menilai jam dan hari lebih representatif untuk memprediksi kemacetan dibandingkan dengan tanggal atau datetime

df['hour'] = df['time'].apply(lambda x : x[11:13]) # Get hours (categorical var)
df['day'] = df['time'].apply(lambda x : datetime.strptime(x[:10], '%Y-%m-%d').strftime('%A')) # Get day
df.head(10)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,avg_length,avg_delay,avg_speed_kmh,total_records,id,date,geometry,hour,day
0,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Van Deventer,3,176.0000,106.000000,4.920000,1,11847,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.61...",16,Friday
1,2019-05-31 23:00:00,32.73,KOTA BANDUNG,HOS Cokroaminoto,3,307.0000,144.000000,6.130000,1,14892,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.59...",23,Friday
2,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Sukaati,2,207.0000,60.000000,6.110000,1,11803,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[107.62...",16,Friday
3,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Lama,3,220.0000,161.500000,3.931000,10,11941,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",17,Friday
4,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Raya,3,1453.4839,475.612900,8.523226,31,11942,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",17,Friday
5,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Raya,4,1266.6395,822.418600,4.809651,86,11943,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",17,Friday
6,2019-05-31 18:00:00,32.73,KOTA BANDUNG,Citayam Raya,2,299.0000,71.000000,8.170000,1,12843,2019-05-31,"{""type"": ""LineString"", ""coordinates"": [[106.80...",18,Friday
7,2019-05-31 18:00:00,32.73,KOTA BANDUNG,Gatot Subroto,2,1217.0602,100.216866,19.438795,83,12882,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",18,Friday
8,2019-05-31 18:00:00,32.73,KOTA BANDUNG,Gatot Subroto,3,770.7159,130.034090,12.962614,88,12883,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",18,Friday
9,2019-05-31 18:00:00,32.73,KOTA BANDUNG,Jalan Jurang,2,625.0000,123.500000,9.976666,6,12934,2019-05-31,"{""type"": ""MultiLineString"", ""coordinates"": [[[...",18,Friday


## Transformasi geometry ke nama Sub Wilayah Kota (SWK)

In [104]:
# Pada project ini saya menggunakan data eksternal dari pemerintah Kota Bandung yaitu data batas-batas Sub Wilayah Kota (SWK).
# Tujuannya adalah untuk memberikan label pada lokasi kemacetan sesuai dengan SWK, alih-alih menggunakan nama jalan.

url = "http://data.bandung.go.id/dataset/fbec5a0e-2efe-4d37-99ee-7a55d32beb20/resource/0a9aefc2-c849-4fb5-b393-b1687dbe8d70/download/3273-kota-bandung-level-kewilayahan.json"
gdf_swk = geopandas.read_file(url)
gdf_swk

,id_wilayah,nama_wilayah,geometry
0,5F142D26,Ujungberung,"POLYGON ((107.71456 -6.89047, 107.71457 -6.890..."
1,CC7B08F0,Tegallega,"POLYGON ((107.56940 -6.91107, 107.56964 -6.911..."
2,BB4C0B50,Kordon,"POLYGON ((107.66345 -6.93120, 107.66400 -6.931..."
3,6CE3653D,Karees,"POLYGON ((107.65666 -6.90218, 107.65700 -6.902..."
4,932659F2,Cibeunying,"POLYGON ((107.60411 -6.84220, 107.60407 -6.842..."
5,7D00F8E4,Bojonagara,"POLYGON ((107.59529 -6.84090, 107.59532 -6.840..."
6,0A471A31,Gedebage,"POLYGON ((107.68166 -6.93780, 107.68166 -6.937..."
7,B3D12B15,Arcamanik,"POLYGON ((107.67346 -6.89192, 107.67349 -6.891..."


In [105]:
# Standardisasikan kolom geometry agar dapat diproses oleh geopandas

df['geometry'] = df['geometry'].apply(lambda x : (json.loads(x)['type'] == 'LineString' and LineString(json.loads(x)['coordinates']) or 
                                                  json.loads(x)['type'] == 'MultiLineString' and MultiLineString(json.loads(x)['coordinates'])))
df.head(5)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,avg_length,avg_delay,avg_speed_kmh,total_records,id,date,geometry,hour,day
0,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Van Deventer,3,176.0000,106.0000,4.920000,1,11847,2019-05-31,"LINESTRING (107.614892 -6.917784, 107.615278 -...",16,Friday
1,2019-05-31 23:00:00,32.73,KOTA BANDUNG,HOS Cokroaminoto,3,307.0000,144.0000,6.130000,1,14892,2019-05-31,"LINESTRING (107.597441 -6.901219, 107.597453 -...",23,Friday
2,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Sukaati,2,207.0000,60.0000,6.110000,1,11803,2019-05-31,"LINESTRING (107.62308 -6.96115, 107.623086 -6....",16,Friday
3,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Lama,3,220.0000,161.5000,3.931000,10,11941,2019-05-31,"(LINESTRING (107.5924 -6.948026, 107.592741 -6...",17,Friday
4,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Raya,3,1453.4839,475.6129,8.523226,31,11942,2019-05-31,"(LINESTRING (107.593294 -6.953205, 107.593288 ...",17,Friday


In [106]:
# Nyatakan lokasi jalan dalam centroid. Centroid digunakan untuk mewakili lokasi kemacetan.

d = {'street': list(df['street']), 'geometry': list(df['geometry'])}
gdf = geopandas.GeoDataFrame(d, crs="EPSG:3395")
gdf['centroid'] = gdf.centroid
gdf.head(5)

,street,geometry,centroid
0,Van Deventer,"LINESTRING (107.61489 -6.91778, 107.61528 -6.9...",POINT (107.61555 -6.91734)
1,HOS Cokroaminoto,"LINESTRING (107.59744 -6.90122, 107.59745 -6.9...",POINT (107.59748 -6.90261)
2,Sukaati,"LINESTRING (107.62308 -6.96115, 107.62309 -6.9...",POINT (107.62304 -6.96022)
3,Cibaduyut Lama,"MULTILINESTRING ((107.59240 -6.94803, 107.5927...",POINT (107.59335 -6.94835)
4,Cibaduyut Raya,"MULTILINESTRING ((107.59329 -6.95320, 107.5932...",POINT (107.59361 -6.95480)


In [107]:
# Cek centroid masuk ke dalam SWK mana

swk_list = list(gdf_swk['nama_wilayah'])
for i in range(len(swk_list)):
  gdf["wilayah_"+swk_list[i]] = gdf["centroid"].within(gdf_swk['geometry'][i]).astype('int')

def get_wilayah(row):
  for c in gdf.iloc[:, 3:].columns:
    if row[c]==1:
      return c[8:]

gdf["wilayah"] = gdf.iloc[:, 3:].apply(get_wilayah, axis=1)
gdf.head(5)

,street,geometry,centroid,wilayah_Ujungberung,wilayah_Tegallega,wilayah_Kordon,wilayah_Karees,wilayah_Cibeunying,wilayah_Bojonagara,wilayah_Gedebage,wilayah_Arcamanik,wilayah
0,Van Deventer,"LINESTRING (107.61489 -6.91778, 107.61528 -6.9...",POINT (107.61555 -6.91734),0,0,0,0,1,0,0,0,Cibeunying
1,HOS Cokroaminoto,"LINESTRING (107.59744 -6.90122, 107.59745 -6.9...",POINT (107.59748 -6.90261),0,0,0,0,0,1,0,0,Bojonagara
2,Sukaati,"LINESTRING (107.62308 -6.96115, 107.62309 -6.9...",POINT (107.62304 -6.96022),0,0,1,0,0,0,0,0,Kordon
3,Cibaduyut Lama,"MULTILINESTRING ((107.59240 -6.94803, 107.5927...",POINT (107.59335 -6.94835),0,1,0,0,0,0,0,0,Tegallega
4,Cibaduyut Raya,"MULTILINESTRING ((107.59329 -6.95320, 107.5932...",POINT (107.59361 -6.95480),0,1,0,0,0,0,0,0,Tegallega


In [108]:
# Jika kolom Wilayah tidak empty, maka tempat obesrvasi tersebut tervalidasi merupakan bagian dari Kota Bandung menurut situs
# http://data.bandung.go.id/dataset/data-spasial-sub-wilayah-kota-bandung/resource/0a9aefc2-c849-4fb5-b393-b1687dbe8d70?view_id=c0d828a7-0aae-4fe8-ae70-591c8ea1f1de
# Pada tahap ini akan dilakukan proses removal pada missing values supaya data benar-benar berisi lokasi kemacetan di Kota Bandung.

for col in gdf.iloc[:, 3:].columns:
  df[col] = gdf[col]

df = df.dropna() # mengeluarkan observasi yang bukan termasuk kota Bandung
df.head(5)

,time,kemendagri_kabupaten_kode,kemendagri_kabupaten_nama,street,level,avg_length,avg_delay,avg_speed_kmh,total_records,id,date,geometry,hour,day,wilayah_Ujungberung,wilayah_Tegallega,wilayah_Kordon,wilayah_Karees,wilayah_Cibeunying,wilayah_Bojonagara,wilayah_Gedebage,wilayah_Arcamanik,wilayah
0,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Van Deventer,3,176.0000,106.0000,4.920000,1,11847,2019-05-31,"LINESTRING (107.614892 -6.917784, 107.615278 -...",16,Friday,0,0,0,0,1,0,0,0,Cibeunying
1,2019-05-31 23:00:00,32.73,KOTA BANDUNG,HOS Cokroaminoto,3,307.0000,144.0000,6.130000,1,14892,2019-05-31,"LINESTRING (107.597441 -6.901219, 107.597453 -...",23,Friday,0,0,0,0,0,1,0,0,Bojonagara
2,2019-05-31 16:00:00,32.73,KOTA BANDUNG,Sukaati,2,207.0000,60.0000,6.110000,1,11803,2019-05-31,"LINESTRING (107.62308 -6.96115, 107.623086 -6....",16,Friday,0,0,1,0,0,0,0,0,Kordon
3,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Lama,3,220.0000,161.5000,3.931000,10,11941,2019-05-31,"(LINESTRING (107.5924 -6.948026, 107.592741 -6...",17,Friday,0,1,0,0,0,0,0,0,Tegallega
4,2019-05-31 17:00:00,32.73,KOTA BANDUNG,Cibaduyut Raya,3,1453.4839,475.6129,8.523226,31,11942,2019-05-31,"(LINESTRING (107.593294 -6.953205, 107.593288 ...",17,Friday,0,1,0,0,0,0,0,0,Tegallega


## Di jam berapa level kemacetan tercatat tinggi?

Akan dilakukan perhitungan observasi kemacetan menurut jam. Hasilnya adalah sebagai berikut:

In [109]:
df[df['level'] == 5].groupby(['hour']).count()['level'].sort_values(ascending=False).head(5)

hour
20    11
17    10
23     9
21     9
19     9
Name: level, dtype: int64

In [110]:
df[df['level'] == 4].groupby(['hour']).count()['level'].sort_values(ascending=False).head(5)

hour
17    1686
16    1471
14    1350
11    1299
13    1224
Name: level, dtype: int64

In [111]:
df[df['level'] == 3].groupby(['hour']).count()['level'].sort_values(ascending=False).head(5)

hour
17    2980
16    2697
14    2496
13    2455
11    2325
Name: level, dtype: int64

In [112]:
df[df['level'] == 2].groupby(['hour']).count()['level'].sort_values(ascending=False).head(5)

hour
17    2644
16    2433
14    2247
13    2232
18    2190
Name: level, dtype: int64

In [113]:
df[df['level'] == 1].groupby(['hour']).count()['level'].sort_values(ascending=False).head(5)

hour
17    1016
18     957
16     939
19     931
14     857
Name: level, dtype: int64

### Insight
* Pukul 5 sore menjadi waktu yang selalu tercatat di semua level kemacetan, disarankan untuk tidak berkendara di jam tersebut.
* Terdapat kelompok jam yang sama pada _top 5 hours_ dari level kemacetan level 3 dan 4, hindari berkendara pada pukul 2 siang hingga 5 sore dan juga pukul 11 siang.

## Di hari dan pukul berapa level kemacetan tercatat tinggi?

Setelah dilakukan perhitungan observasi kemacetan menurut jam, pada bagian ini akan ditambah faktor hari. Hasilnya adalah sebagai berikut:

In [114]:
df[df['level'] == 5].groupby(['day', 'hour']).count()['level'].sort_values(ascending=False).head(5)

day       hour
Saturday  17      4
          20      3
Sunday    21      2
          18      2
Saturday  16      2
Name: level, dtype: int64

In [115]:
df[df['level'] == 4].groupby(['day', 'hour']).count()['level'].sort_values(ascending=False).head(5)

day       hour
Friday    17      333
Saturday  17      269
          16      264
Friday    16      259
Saturday  14      259
Name: level, dtype: int64

In [116]:
df[df['level'] == 3].groupby(['day', 'hour']).count()['level'].sort_values(ascending=False).head(5)

day       hour
Friday    17      520
Monday    17      470
Saturday  16      466
          17      452
          14      429
Name: level, dtype: int64

In [117]:
df[df['level'] == 2].groupby(['day', 'hour']).count()['level'].sort_values(ascending=False).head(5)

day       hour
Friday    17      424
Saturday  17      418
Monday    16      401
          17      397
Saturday  16      389
Name: level, dtype: int64

In [118]:
df[df['level'] == 1].groupby(['day', 'hour']).count()['level'].sort_values(ascending=False).head(5)

day       hour
Saturday  17      184
          19      162
          16      158
Friday    18      151
Saturday  18      151
Name: level, dtype: int64

### Insight

* _Top 5 hours_ pada level kemacetan level 5 terjadi pada akhir pekan di kisaran pukul 4 hingga 6 sore dan pukul 8 hingga 9 malam, hindari berkendara pada waktu-waktu tersebut.
* Hari Jumat pukul 5 sore mencatatkan level kemacetan level 2-4 dengan jumlah kejadian tertinggi.
* Pada level kemacetan level 1, jumlah terbanyak ditunjukkan pada hari Sabtu pukul 4 sore hingga 7 malam, mengindikasikan bahwa level kemacetan cenderung menurun di sore menjelang malam.

## Analisis lanjutan: Wilayah Kemacetan

Analisis lanjutan dari jam dan hari dengan memasukkan faktor wilayah. Hasilnya adalah sebagai berikut:

In [119]:
df[df['level'] == 5].groupby(['day', 'hour', 'wilayah']).count()['level'].sort_values(ascending=False).head(5)

day       hour  wilayah   
Saturday  17    Cibeunying    3
Sunday    20    Cibeunying    2
          16    Cibeunying    2
Saturday  16    Cibeunying    2
          18    Cibeunying    2
Name: level, dtype: int64

In [120]:
df[df['level'] == 4].groupby(['day', 'hour', 'wilayah']).count()['level'].sort_values(ascending=False).head(5)

day       hour  wilayah   
Friday    17    Cibeunying    96
Saturday  14    Cibeunying    93
          20    Cibeunying    88
          19    Cibeunying    88
Friday    17    Bojonagara    87
Name: level, dtype: int64

In [121]:
df[df['level'] == 3].groupby(['day', 'hour', 'wilayah']).count()['level'].sort_values(ascending=False).head(5)

day       hour  wilayah   
Friday    17    Cibeunying    166
Saturday  14    Cibeunying    152
          19    Cibeunying    148
          13    Cibeunying    147
Monday    14    Cibeunying    147
Name: level, dtype: int64

In [122]:
df[df['level'] == 2].groupby(['day', 'hour', 'wilayah']).count()['level'].sort_values(ascending=False).head(5)

day       hour  wilayah   
Friday    17    Cibeunying    138
Saturday  19    Cibeunying    127
          14    Cibeunying    125
          20    Cibeunying    124
Monday    13    Cibeunying    122
Name: level, dtype: int64

In [123]:
df[df['level'] == 1].groupby(['day', 'hour', 'wilayah']).count()['level'].sort_values(ascending=False).head(5)

day       hour  wilayah   
Saturday  17    Cibeunying    60
Sunday    17    Cibeunying    52
Friday    18    Cibeunying    48
          17    Cibeunying    48
Saturday  13    Cibeunying    47
Name: level, dtype: int64

### Insight

* Untuk semua tingkat kemacetan, terlihat bahwa Sub Wilayah Kota (SWK) Cibeunying mencatatkan top 5 jumlah kejadian terbanyak kecuali untuk tingkat kemacetan level 4.
* Jumat dan Sabtu pukul 5 sore merupakan waktu dengan kejadian terbanyak untuk semua tingkat kemacetan, hindari berkendara di daerah Cibeunying pada waktu-waktu tersebut.

# Modelling

## Memilih Fitur

In [124]:
df.columns

Index(['time', 'kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama',
       'street', 'level', 'avg_length', 'avg_delay', 'avg_speed_kmh',
       'total_records', 'id', 'date', 'geometry', 'hour', 'day',
       'wilayah_Ujungberung', 'wilayah_Tegallega', 'wilayah_Kordon',
       'wilayah_Karees', 'wilayah_Cibeunying', 'wilayah_Bojonagara',
       'wilayah_Gedebage', 'wilayah_Arcamanik', 'wilayah'],
      dtype='object')

In [125]:
# Pada tahap ini akan diseleksi fitur-fitur yang diperlukan untuk membuat model

df_model = df.drop(['kemendagri_kabupaten_kode', 'kemendagri_kabupaten_nama'], axis=1) # drop fitur yang memiliki value sama karena tidak berdampak pada model
df_model = df_model.drop(['time', 'street', 'geometry', 'wilayah'], axis=1) # drop fitur yang sudah ditransformasi di tahap Exploratory Data Analysis
df_model = df_model.drop(['total_records', 'id', 'date'], axis=1) # drop fitur yang dinilai tidak akan berkontribusi pada model
df_model.head(5)

,level,avg_length,avg_delay,avg_speed_kmh,hour,day,wilayah_Ujungberung,wilayah_Tegallega,wilayah_Kordon,wilayah_Karees,wilayah_Cibeunying,wilayah_Bojonagara,wilayah_Gedebage,wilayah_Arcamanik
0,3,176.0000,106.0000,4.920000,16,Friday,0,0,0,0,1,0,0,0
1,3,307.0000,144.0000,6.130000,23,Friday,0,0,0,0,0,1,0,0
2,2,207.0000,60.0000,6.110000,16,Friday,0,0,1,0,0,0,0,0
3,3,220.0000,161.5000,3.931000,17,Friday,0,1,0,0,0,0,0,0
4,3,1453.4839,475.6129,8.523226,17,Friday,0,1,0,0,0,0,0,0


## Encode Fitur Kategorik

In [126]:
# Pada pemodelan Machine Learning ini, jam dan hari akan dilihat sebagai fitur kategorik, sehingga perlu dilakukan encoding

def create_dummy_df(df, cat_cols, dummy_na):
    for col in cat_cols:
        try:
            df = pd.concat([df.drop(col, axis=1), pd.get_dummies(df[col], prefix=col, prefix_sep='_', dummy_na=dummy_na)], axis=1)
        except:
            continue

    return df

cat_df = df_model.select_dtypes(include=['object'])
cat_cols_lst = cat_df.columns

df_model = create_dummy_df(df_model, cat_cols_lst, dummy_na=False)
df_model.head(5)

,level,avg_length,avg_delay,avg_speed_kmh,wilayah_Ujungberung,wilayah_Tegallega,wilayah_Kordon,wilayah_Karees,wilayah_Cibeunying,wilayah_Bojonagara,wilayah_Gedebage,wilayah_Arcamanik,hour_00,hour_01,hour_02,hour_03,hour_04,hour_05,hour_06,hour_07,hour_08,hour_09,hour_10,hour_11,hour_12,hour_13,hour_14,hour_15,hour_16,hour_17,hour_18,hour_19,hour_20,hour_21,hour_22,hour_23,day_Friday,day_Monday,day_Saturday,day_Sunday,day_Thursday,day_Tuesday,day_Wednesday
0,3,176.0000,106.0000,4.920000,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1,3,307.0000,144.0000,6.130000,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0
2,2,207.0000,60.0000,6.110000,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,220.0000,161.5000,3.931000,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0
4,3,1453.4839,475.6129,8.523226,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0


## Pemisahan Data Training dan Testing

In [127]:
# Pemisahan data training dan data testing

from sklearn.model_selection import train_test_split

y = df_model['level']
X = df_model.iloc[:,1:]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

## Multiclass Classifier Model



In [128]:
# Variabel target y yaitu level kemacetan dilihat sebagai variabel kategorik. Pemodelan yang akan diterapkan adalah pemodelan klasifikasi multiclass
# karena level kemacetan merupakan bilangan bulat dari 1-5. Akan dicari model terbaik dari strategi pemodelan multiclass 
# One vs Rest dan One vs One Classifier. Dengan menggunakan model-model Logistic Regression, Decision Tree Classifier 
# dan Random Forest Classifier, berikut adalah pemodelannya.

# Model One vs Rest (OVR Classifier)

from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# Instantiate dan Fit
ovr_log = OneVsRestClassifier(LogisticRegression()).fit(X_train, y_train)
ovr_dt = OneVsRestClassifier(DecisionTreeClassifier()).fit(X_train, y_train)
ovr_rf = OneVsRestClassifier(RandomForestClassifier()).fit(X_train, y_train)

# Prediksi
y_test_pred_log = ovr_log.predict(X_test)
y_test_pred_dt = ovr_dt.predict(X_test)
y_test_pred_rf = ovr_rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


In [129]:
# Model One vs One (OVO Classifier)

from sklearn.multiclass import OneVsOneClassifier

# Instantiate dan Fit
ovo_log = OneVsOneClassifier(LogisticRegression()).fit(X_train, y_train)
ovo_dt = OneVsOneClassifier(DecisionTreeClassifier()).fit(X_train, y_train)
ovo_rf = OneVsOneClassifier(RandomForestClassifier()).fit(X_train, y_train)

# Prediksi
y_test_pred_log_ovo = ovo_log.predict(X_test)
y_test_pred_dt_ovo = ovo_dt.predict(X_test)
y_test_pred_rf_ovo = ovo_rf.predict(X_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression


## Memilih Model

In [130]:
# Pada tahap ini akan dibandingkan akurasi dan nilai r-squared dari masing-masing model

# Score OVR
## Logistic OVR
print("R-Square Logisitc OVR: " + str(r2_score(y_test, y_test_pred_log)))
print("Accuracy Logisitc OVR: " + str(round(accuracy_score(y_test, y_test_pred_log)*100,2)) + "%")
## Decision Tree OVR
print("R-Square Decision Tree OVR: " + str(r2_score(y_test, y_test_pred_dt)))
print("Accuracy Decision Tree OVR: " + str(round(accuracy_score(y_test, y_test_pred_dt)*100,2)) + "%")
## Random Forest OVR
print("R-Square Random Forest OVR: " + str(r2_score(y_test, y_test_pred_rf)))
print("Accuracy Random Forest OVR: " + str(round(accuracy_score(y_test, y_test_pred_rf)*100,2)) + "%")

R-Square Logisitc OVR: 0.6016210820249503
Accuracy Logisitc OVR: 69.98%
R-Square Decision Tree OVR: -0.19255393546708643
Accuracy Decision Tree OVR: 70.43%
R-Square Random Forest OVR: 0.7023926602516204
Accuracy Random Forest OVR: 77.98%


In [131]:
# Score OVO
## Logistic OVO
print("R-Square Logisitc OVO: " + str(r2_score(y_test, y_test_pred_log_ovo)))
print("Accuracy Logisitc OVO: " + str(round(accuracy_score(y_test, y_test_pred_log_ovo)*100,2)) + "%")
## Decision Tree OVO
print("R-Square Decision Tree OVO: " + str(r2_score(y_test, y_test_pred_dt_ovo)))
print("Accuracy Decision Tree OVO: " + str(round(accuracy_score(y_test, y_test_pred_dt_ovo)*100,2)) + "%")
## Random Forest OVO
print("R-Square Random Forest OVO: " + str(r2_score(y_test, y_test_pred_rf_ovo)))
print("Accuracy Random Forest OVO: " + str(round(accuracy_score(y_test, y_test_pred_rf_ovo)*100,2)) + "%")

R-Square Logisitc OVO: 0.6696908459035691
Accuracy Logisitc OVO: 75.36%
R-Square Decision Tree OVO: 0.6466537715921394
Accuracy Decision Tree OVO: 75.64%
R-Square Random Forest OVO: 0.7075582971677457
Accuracy Random Forest OVO: 78.51%


Dilihat dari nilai R-Square dan akurasi model, maka dipilih **model Random Forest Classifier dengan strategi One vs One Classifier** dengan nilai R-square 0.707 dan nilai akurasi 78.51%.

## Melakukan k-Fold Cross Validation pada Model yang Dipilih

In [132]:
# Pada tahap ini dilakukan validasi model untuk melihat apakah model overfit atau underfit

from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = ovo_rf, X = X_train, y = y_train, cv = 10)
print("Accuracy: {:.2f} %".format(accuracies.mean()*100))
print("Standard Deviation: {:.2f} %".format(accuracies.std()*100))

Accuracy: 77.89 %
Standard Deviation: 0.48 %


Dengan selisih nilai akurasi sebesar 0.62% dari validasi dan model yang dipilih, maka dapat disimpulkan bahwa model sudah cukup baik.



## Mengukur Pengaruh Fitur dengan Permutation Feature Importance

In [133]:
# Pada tahap ini diterapkan metode Permutation Feature Importance untuk mengukur fitur mana yang paling memengaruhi model yang dipilih

from rfpimp import permutation_importances

def r2(rf, X_train, y_train):
    return r2_score(y_train, ovo_rf.predict(X_train))

perm_imp_rfpimp = permutation_importances(ovo_rf, X_train, y_train, r2)

In [134]:
perm_imp_rfpimp.sort_values

<bound method DataFrame.sort_values of                      Importance
Feature                        
avg_delay              1.140092
avg_speed_kmh          0.551702
avg_length             0.546626
wilayah_Cibeunying     0.072914
wilayah_Karees         0.072222
wilayah_Bojonagara     0.049840
wilayah_Tegallega      0.044995
day_Wednesday          0.027458
day_Saturday           0.027227
day_Friday             0.026997
day_Thursday           0.026305
day_Sunday             0.025843
day_Tuesday            0.021920
day_Monday             0.020767
hour_17                0.018921
wilayah_Kordon         0.016844
hour_11                0.016613
hour_18                0.015921
hour_14                0.015690
hour_20                0.015460
hour_15                0.013844
hour_16                0.013383
hour_12                0.012229
hour_13                0.011768
hour_10                0.011076
hour_19                0.008768
wilayah_Ujungberung    0.008537
wilayah_Arcamanik      0.008537
h

### Insight

* Rata-rata waktu _delay_ (dalam detik) yang diakibatkan oleh kemacetan menjadi faktor yang paling berpengaruh pada level kemacetan.
* Kecepatan rata-rata dan panjang antrian menjadi dua faktor yang paling berpengaruh berikutnya pada level kemacetan.
* Keberadaan kendaraan di wilayah Cibeunying, Karees dan Bojonagara menjadi tiga wilayah teratas yang berpengaruh pada level kemacetan.
* Hari Rabu, Sabtu dan Jumat merupakan tiga hari teratas yang berpengaruh pada level kemacetan.
* Pukul 5 sore menjadi waktu yang paling berpengaruh pada level kemacetan.

# Kesimpulan dan Rekomendasi

**Sesuai dengan _insights_ yang telah dipaparkan, maka poin penting yang dapat diambil adalah:**

* **Hindari berkendara pada pukul 5 sore terutama pada hari Jumat dan Sabtu** karena telah ditunjukkan pada Exploratory Data Analysis bahwa pukul 5 sore merupakan waktu yang selalu tercatat dalam _top 5_ di semua level kemacetan. Selain itu, pengaruh fitur pukul 5 sore pada model juga menunjukkan pengaruh yang paling kuat dibanding jam-jam lainnya.
* **Hindari khususnya Kawasan Cibeunying** pada waktu yang telah disebutkan di poin pertama karena kemacetan di Cibeunying tercatat paling banyak untuk semua level kemacetan. Selain itu, pengaruh fitur wilayah Cibeunying juga menunjukkan pengaruh paling kuat dibanding wilayah-wilayah lainnya.
* **Rata-rata waktu _delay_ menjadi faktor yang paling memengaruhi level kemacetan**, disarankan untuk mengambil jalan alternatif atau menunggu hingga malam jika mengalami _delay_ yang cukup panjang.